In [1]:
import pandas as pd
import numpy as np

In [2]:
#pip install google-adk
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext, FunctionTool
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.models.google_llm import Gemini

In [3]:
import APIKey    #Storing my API key in python class "APIKey"
import os

google_API = APIKey.GOOGLESTUDIO_API_KEY    #My google studio API

os.environ["GOOGLE_API_KEY"] = google_API

In [15]:
import yfinance as yf

ticker = "SHEL"
shell = yf.Ticker(ticker)

shell_stock_price = yf.download(ticker, period = "5y", auto_adjust=False)
shell_stock_price

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,SHEL,SHEL,SHEL,SHEL,SHEL,SHEL
Date,,,,,,
2020-12-07,31.050993,37.419998,38.000000,37.334999,37.919998,4176276
2020-12-08,31.026100,37.389999,37.810001,37.209999,37.290001,4120911
2020-12-09,31.183773,37.580002,38.325001,37.110001,38.139999,6450576
2020-12-10,31.880795,38.419998,38.869999,37.619999,37.720001,4965517
2020-12-11,31.374619,37.810001,37.970001,37.160000,37.950001,4557049
...,...,...,...,...,...,...
2025-12-01,74.309998,74.309998,74.894997,74.139999,74.180000,3304283
2025-12-02,73.959999,73.959999,74.320999,73.570000,74.230003,3716093


# Info

In [5]:
info = shell.info
info

{'address1': 'Shell Centre',
 'city': 'London',
 'zip': 'SE1 7NA',
 'country': 'United Kingdom',
 'phone': '44 20 7934 1234',
 'website': 'https://www.shell.com',
 'industry': 'Oil & Gas Integrated',
 'industryKey': 'oil-gas-integrated',
 'industryDisp': 'Oil & Gas Integrated',
 'sector': 'Energy',
 'sectorKey': 'energy',
 'sectorDisp': 'Energy',
 'longBusinessSummary': 'Shell plc operates as an energy and petrochemical company Europe, Asia, Oceania, Africa, the United States, and other Americas. It operates through Integrated Gas; Upstream; Marketing; Chemicals and Products; and Renewables and Energy Solutions segments. The company explores for and extracts natural gas to produce liquefied natural gas or convert into gas-to-liquids products; explores for and extracts crude oil and natural gas liquids; and operates upstream and midstream infrastructure to deliver gas to market. It is also involved in marketing supplies fuels and lubricants for transport, manufacturing, mining, power ge

In [6]:
# Using summary as part of the Ai prompt
summary = info['longBusinessSummary']
summary

'Shell plc operates as an energy and petrochemical company Europe, Asia, Oceania, Africa, the United States, and other Americas. It operates through Integrated Gas; Upstream; Marketing; Chemicals and Products; and Renewables and Energy Solutions segments. The company explores for and extracts natural gas to produce liquefied natural gas or convert into gas-to-liquids products; explores for and extracts crude oil and natural gas liquids; and operates upstream and midstream infrastructure to deliver gas to market. It is also involved in marketing supplies fuels and lubricants for transport, manufacturing, mining, power generation, agriculture, and construction industries; operates electric vehicle charging and convenience retail; turn crude oil and other feedstocks into products for households, industry, and transport; trades crude oil, oil products, and petrochemicals; and oil sand activities. In addition, the company generates, markets, and trades power from wind, solar and pipeline ga

In [23]:
# Comparing some financial data with its competitors 
compare_df = pd.DataFrame(columns = ['Ticker', 'Dividend Yield', 'Trailing PE', 'TTM PS', 'Profit Margin', 'PB Ratio', 'Trailing EPS', 'EV/EBITDA', 
                                     'Current Ratio', 'Debt-to-Equity', 'ROA', 'ROE', 'PEG Ratio'])

shell_list = [ticker, info['dividendYield'], info['trailingPE'], info['priceToSalesTrailing12Months'], info['profitMargins'], info['priceToBook'], 
             info['trailingEps'], info['enterpriseToEbitda'], info['currentRatio'], info['debtToEquity'], info['returnOnAssets'],
             info['returnOnEquity'], info['trailingPegRatio']] 

compare_df.loc[0] = shell_list
compare_df

,Ticker,Dividend Yield,Trailing PE,TTM PS,Profit Margin,PB Ratio,Trailing EPS,EV/EBITDA,Current Ratio,Debt-to-Equity,ROA,ROE,PEG Ratio
0,SHEL,3.84,15.073469,0.784398,0.05438,2.441249,4.9,9.761,1.352,41.602,0.04901,0.08155,1.6122


In [17]:
# Find the companies' main competitors
industry = yf.Industry(info['industryKey'])

competitors = list(industry.top_companies.index.values)
competitors

['XOM', 'CVX', 'NFG', 'DEC', 'UNTC']

In [26]:
competitors.insert(0, ticker)
competitors

['SHEL', 'XOM', 'CVX', 'NFG', 'DEC', 'UNTC']

In [24]:
metrics = [
    'dividendYield', 'trailingPE', 'priceToSalesTrailing12Months', 'profitMargins', 'priceToBook', 'trailingEps', 'enterpriseToEbitda', 
    'currentRatio', 'debtToEquity', 'returnOnAssets', 'returnOnEquity', 'trailingPegRatio'
]

for comp in competitors:
    comp_info = yf.Ticker(comp).info
    
    # Start the row with the company name
    row_data = [comp]
    
    # Loop through your metrics list to fetch data safely
    for metric in metrics:
        # returns "NA" if key is missing
        value = comp_info.get(metric, 'NaN') 
        row_data.append(value)
    
    compare_df.loc[len(compare_df)] = row_data

compare_df = compare_df.replace("NaN", np.nan)
compare_df = compare_df.dropna(axis=1, how='all')
compare_df = compare_df.fillna("N/A")

compare_df

C:\Users\User\AppData\Local\Temp\ipykernel_3124\2192629104.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  compare_df.loc[len(compare_df)] = row_data
C:\Users\User\AppData\Local\Temp\ipykernel_3124\2192629104.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  compare_df.loc[len(compare_df)] = row_data
C:\Users\User\AppData\Local\Temp\ipykernel_3124\2192629104.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this

,Ticker,Dividend Yield,Trailing PE,TTM PS,Profit Margin,PB Ratio,Trailing EPS,EV/EBITDA,Current Ratio,Debt-to-Equity,ROA,ROE,PEG Ratio
0,SHEL,3.84,15.073469,0.784398,0.05438,2.441249,4.90,9.761,1.352,41.602,0.04901,0.08155,1.6122
1,XOM,3.52,17.109737,1.538255,0.09181,1.905205,6.88,8.589,1.137,15.672,0.05108,0.11424,2.7746
2,CVX,4.49,21.407877,1.625317,0.06767,1.614378,7.11,9.186,1.152,21.239,0.04047,0.07316,2.911
3,NFG,2.60,14.495599,3.267546,0.22766,2.404644,5.68,7.106,0.444,92.879,0.07244,0.17449,N/A
4,DEC,7.57,N/A,1.101139,-0.12000,1.736562,-2.66,11.127,0.537,379.369,0.00406,-0.21422,N/A
5,UNTC,15.15,4.896142,1.319219,0.23337,1.346554,6.74,2.891,3.545,1.009,0.14997,0.26941,N/A


In [10]:
# Round the number for a more effective AI agent
compare_df = compare_df.round(4)

# Convert the dataframe to string for AI prompt
compare_df_string = compare_df.to_markdown(index=False)
compare_df_string

'| Ticker   |   Dividend Yield |   Trailing PE |   TTM PS |   Profit Margin |   PB Ratio |   Trailing EPS |   EV/EBITDA |   Current Ratio |   Debt-to-Equity |    ROA |    ROE |   PEG Ratio |\n|:---------|-----------------:|--------------:|---------:|----------------:|-----------:|---------------:|------------:|----------------:|-----------------:|-------:|-------:|------------:|\n| SHEL     |             3.8  |       15.3694 |   0.8023 |          0.0544 |     2.4892 |           4.9  |       9.868 |           1.352 |           41.602 | 0.049  | 0.0815 |      1.634  |\n| XOM      |             3.5  |       17.3009 |   1.5554 |          0.0918 |     1.9265 |           6.88 |       8.626 |           1.137 |           15.672 | 0.0511 | 0.1142 |      2.7756 |\n| CVX      |             4.42 |       21.6062 |   1.6404 |          0.0677 |     1.6293 |           7.11 |       9.323 |           1.152 |           21.239 | 0.0405 | 0.0732 |      2.9249 |\n| NFG      |             2.66 |       14.318

# News AI Agent 
### With news generate by google search

In [6]:
google_news_agent = Agent(
    name="GoogleNewsAgent",
    model="gemini-2.5-flash",
    instruction="""You are a specialized news arrangement agent for a given company. 
    
    For a given company:
    1. Use 'google_search' to find the company's summary.
    2. Read the company's summary and then use 'google_search' to find current related news that would affect the company's stock price.
    3. Generate comment with important points and analysis about the company's stock future performance base on the news.
    """,
    tools = [google_search],
    output_key="google_news_arrangement",
)

In [12]:
runner = InMemoryRunner(agent=google_news_agent)
prompt = f"""{ticker} summary: 

{summary}"""
response = await runner.run_debug(prompt)


 ### Created new session: debug_session_id

User > SHEL summary: 

Shell plc operates as an energy and petrochemical company Europe, Asia, Oceania, Africa, the United States, and other Americas. It operates through Integrated Gas; Upstream; Marketing; Chemicals and Products; and Renewables and Energy Solutions segments. The company explores for and extracts natural gas to produce liquefied natural gas or convert into gas-to-liquids products; explores for and extracts crude oil and natural gas liquids; and operates upstream and midstream infrastructure to deliver gas to market. It is also involved in marketing supplies fuels and lubricants for transport, manufacturing, mining, power generation, agriculture, and construction industries; operates electric vehicle charging and convenience retail; turn crude oil and other feedstocks into products for households, industry, and transport; trades crude oil, oil products, and petrochemicals; and oil sand activities. In addition, the company ge

# Comparing AI Agent

In [7]:
competitors_agent = Agent(
    name="CompetitorsAgent",
    model="gemini-2.5-flash",
    instruction="""You are a specialized agent to compare the target company to its competitors.

    For a given company:
    1. Use 'competitors_compare' to get the target company and its main competitors' financial ratio data.
    2. Check the "status" field in it response for errors.
    3. Aanlyse the financial ratio data, summarize the most important points and provide some insights that might affect the target companies' 
    stock price.
    
    If any tool returns status "error", explain the issue to the user clearly.""",
    tools = [competitors_compare],
    output_key="comparing_competitors",
)

In [14]:
runner = InMemoryRunner(agent=competitors_agent)
prompt = f""" Please analyze the following financial data for SHEL and its competitors:

{compare_df_string}"""
response = await runner.run_debug(prompt)


 ### Created new session: debug_session_id

User >  Please analyze the following financial data for SHEL and its competitors:

| Ticker   |   Dividend Yield |   Trailing PE |   TTM PS |   Profit Margin |   PB Ratio |   Trailing EPS |   EV/EBITDA |   Current Ratio |   Debt-to-Equity |    ROA |    ROE |   PEG Ratio |
|:---------|-----------------:|--------------:|---------:|----------------:|-----------:|---------------:|------------:|----------------:|-----------------:|-------:|-------:|------------:|
| SHEL     |             3.8  |       15.3694 |   0.8023 |          0.0544 |     2.4892 |           4.9  |       9.868 |           1.352 |           41.602 | 0.049  | 0.0815 |      1.634  |
| XOM      |             3.5  |       17.3009 |   1.5554 |          0.0918 |     1.9265 |           6.88 |       8.626 |           1.137 |           15.672 | 0.0511 | 0.1142 |      2.7756 |
| CVX      |             4.42 |       21.6062 |   1.6404 |          0.0677 |     1.6293 |           7.11 |      

## Aggregator Agent

In [8]:
aggregator_agent = Agent(
    name="AggregatorAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Combine these two analyses into a single investment memo:

    **News:**
    {google_news_arrangement}
    
    **Comparison:**
    {comparing_competitors}
    
    Your memo should highlight influences to the target stock price, recommendations, and the risk from these two analyses. The final summary should be around 200 words.""",
    output_key="investment_memo", # This will be the final output of the entire system.
)

In [9]:
parallel_analysis_team = ParallelAgent(
    name="ParallelAnalysisTeam",
    sub_agents=[google_news_agent, competitors_agent],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="AnalysisSystem",
    sub_agents=[parallel_analysis_team, aggregator_agent],
)

In [10]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("SHEL")


 ### Created new session: debug_session_id

User > SHEL


C:\Users\User\AppData\Local\Temp\ipykernel_15692\1773593920.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  compare_df.loc[len(compare_df)] = comp_list
C:\Users\User\AppData\Local\Temp\ipykernel_15692\1773593920.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  compare_df.loc[len(compare_df)] = comp_list


GoogleNewsAgent > Shell plc (SHEL) is demonstrating a dual strategy of continued investment in its traditional oil and gas segments while selectively progressing its renewable energy and decarbonization initiatives, all while maintaining a strong focus on shareholder returns. Recent news from late 2025 highlights several key developments that will likely influence the company's stock performance.

**Key Points and Analysis:**

*   **Robust Shareholder Returns:** Shell continues to prioritize shareholder returns, as evidenced by its Q3 2025 interim dividend of $0.358 per ordinary share and an ongoing share buy-back program. The company aims for an approximate 4% annual dividend growth and targets 40-50% of cash flow from operations for shareholder distributions. This consistent return of capital, coupled with a current dividend yield of around 3.9%, generally appeals to income-focused investors and can provide a floor for the stock price.
*   **Mixed Financial Performance:** While Shell